In [22]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_raw = "Resources/schools_complete.csv"
student_raw = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
scdf = pd.read_csv(school_raw)
stdf = pd.read_csv(student_raw)

# Combine the data into a single dataset
cdf = pd.merge(scdf, stdf, how="left", on=["school_name", "school_name"])

ccdf = cdf[['reading_score', 'math_score']]
ccdf = pd.DataFrame(ccdf)
ccdf['bps'] = cdf['budget'] / cdf['size']
ccdf['pm'] = cdf['math_score'] >= 70
ccdf['pr'] = cdf['reading_score'] >= 70

# Sample bins. Feel free to create your own bins.
bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
ccdf['Spending Ranges(Per Student)'] = pd.cut(ccdf["bps"], bins, labels=group_names)

gbo = ccdf.groupby("Spending Ranges(Per Student)")

gcdf = gbo.agg({  "reading_score":"mean",
                  "math_score":"mean", 
                  "pm":"sum", 
                  "pr":"sum"
               })
gcdf['tc'] = gbo['bps'].count()

gcdf['% Passing Math'] = gcdf.pm / gcdf.tc
gcdf['% Passing Reading'] = gcdf.pr / gcdf.tc
gcdf['% Overall Passing Rate'] = (gcdf['% Passing Reading'] + gcdf['% Passing Math']) / 2

fdf = gcdf.rename(columns={'reading_score':'Average Reading Score',
                           'math_score':'Average Math Score'})

fdf = fdf[[ 'Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

fdf.style

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges(Per Student),,,,,
<$585,83.3631,83.964,0.937029,0.966866,0.951947
$585-615,83.5292,83.8384,0.941241,0.958869,0.950055
$615-645,78.0616,81.4341,0.714004,0.836148,0.775076
$645-675,77.0493,81.0056,0.662308,0.811094,0.736701
